# 1. 문자열

- 텐서 → 자연어 처리에 특히 유용한 바이트 문자열 담기 가능

# 2. 래그드 텐서(Ragged Tensor)

- 크기가 다른 배열의 리스트를 표현하는 특별한 종류의 텐서
- 하나 이상의 래그드 차원을 가진 텐서

# 3. 희소 텐서

- 대부분 0으로 채워진 텐서

# 4. 텐서 배열

- 텐서 리스트 표현
- 배열의 모든 위치에서 텐서 읽기/쓰기 가능

# 5. 집합

- 정수, 문자열 집합 지원

# 6. 큐

- 데이터 레코드를 넣고 나중에 추출하는 데이터 구조
- 데이터 적재 및 파이프라인 구현 시 유용
- PaddingFIFOQueue
- PriorityQueue
- RandomShuffleQueue

In [1]:
import tensorflow as tf

tf.constant(b"Hello world")

<tf.Tensor: shape=(), dtype=string, numpy=b'Hello world'>

In [2]:
tf.constant("café")

<tf.Tensor: shape=(), dtype=string, numpy=b'caf\xc3\xa9'>

In [3]:
u = tf.constant([ord(c) for c in "café"])
u

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([ 99,  97, 102, 233], dtype=int32)>

In [4]:
b = tf.strings.unicode_encode(u, "UTF-8")
b

<tf.Tensor: shape=(), dtype=string, numpy=b'caf\xc3\xa9'>

In [5]:
tf.strings.length(b, unit="UTF8_CHAR")

<tf.Tensor: shape=(), dtype=int32, numpy=4>

In [7]:
tf.strings.unicode_decode(b, "UTF-8")

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([ 99,  97, 102, 233], dtype=int32)>

In [8]:
p = tf.constant(["café", "Coffee", "caffe"])
tf.strings.length(p, unit="UTF8_CHAR")

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([4, 6, 5], dtype=int32)>

In [11]:
r = tf.strings.unicode_decode(p, "UTF-8")
r

<tf.RaggedTensor [[99, 97, 102, 233], [67, 111, 102, 102, 101, 101],
 [99, 97, 102, 102, 101]]>

In [12]:
r[1]

<tf.Tensor: shape=(6,), dtype=int32, numpy=array([ 67, 111, 102, 102, 101, 101], dtype=int32)>

In [13]:
r2 = tf.ragged.constant([[65, 66], [], [67]])
tf.concat([r, r2], axis=0)

<tf.RaggedTensor [[99, 97, 102, 233], [67, 111, 102, 102, 101, 101],
 [99, 97, 102, 102, 101], [65, 66], [], [67]]>

In [15]:
r3 = tf.ragged.constant([[68, 69, 70], [71], [], [72, 73]])
print(tf.concat([r, r3], axis=1))

InvalidArgumentError: Input tensors at index 0 (=x) and 1 (=y) have incompatible shapes.
Condition x == y did not hold.
First 1 elements of x:
[3]
First 1 elements of y:
[4]

In [16]:
r.to_tensor()

<tf.Tensor: shape=(3, 6), dtype=int32, numpy=
array([[ 99,  97, 102, 233,   0,   0],
       [ 67, 111, 102, 102, 101, 101],
       [ 99,  97, 102, 102, 101,   0]], dtype=int32)>

In [17]:
s = tf.SparseTensor(indices=[[0, 1], [1, 0], [2, 3]], values=[1., 2., 3.], dense_shape=[3, 4])
tf.sparse.to_dense(s)

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0., 1., 0., 0.],
       [2., 0., 0., 0.],
       [0., 0., 0., 3.]], dtype=float32)>

In [18]:
s * 42.0

SparseTensor(indices=tf.Tensor(
[[0 1]
 [1 0]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([ 42.  84. 126.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))

In [19]:
s + 42.0

TypeError: unsupported operand type(s) for +: 'SparseTensor' and 'float'

In [20]:
array = tf.TensorArray(dtype=tf.float32, size=3)
array = array.write(0, tf.constant([1., 2.]))
array = array.write(1, tf.constant([3., 10.]))
array = array.write(2, tf.constant([5., 7.]))
tensor1 = array.read(1)

In [21]:
array.stack()

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[1., 2.],
       [0., 0.],
       [5., 7.]], dtype=float32)>

In [22]:
a = tf.constant([[1, 5, 9]])
b = tf.constant([[5, 6, 9, 11]])
u = tf.sets.union(a, b)
u

SparseTensor(indices=tf.Tensor(
[[0 0]
 [0 1]
 [0 2]
 [0 3]
 [0 4]], shape=(5, 2), dtype=int64), values=tf.Tensor([ 1  5  6  9 11], shape=(5,), dtype=int32), dense_shape=tf.Tensor([1 5], shape=(2,), dtype=int64))

In [23]:
tf.sparse.to_dense(u)

<tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 1,  5,  6,  9, 11]], dtype=int32)>

In [24]:
a = tf.constant([[1, 5, 9], [10, 0, 0]])
b = tf.constant([[5, 6, 9, 11], [13, 0, 0, 0]])
u = tf.sets.union(a, b)
tf.sparse.to_dense(u)

<tf.Tensor: shape=(2, 5), dtype=int32, numpy=
array([[ 1,  5,  6,  9, 11],
       [ 0, 10, 13,  0,  0]], dtype=int32)>

In [25]:
q = tf.queue.FIFOQueue(3, [tf.int32, tf.string], shapes=[(), ()])
q.enqueue([10, b"windy"])
q.enqueue([15, b"sunny"])
q.size()

<tf.Tensor: shape=(), dtype=int32, numpy=2>

In [26]:
q.dequeue()

[<tf.Tensor: shape=(), dtype=int32, numpy=10>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'windy'>]

In [27]:
q.enqueue_many([[13, 16], [b"cloudy", b"rainy"]])
q.dequeue_many(3)

[<tf.Tensor: shape=(3,), dtype=int32, numpy=array([15, 13, 16], dtype=int32)>,
 <tf.Tensor: shape=(3,), dtype=string, numpy=array([b'sunny', b'cloudy', b'rainy'], dtype=object)>]